# Ввведение в байесовскую статистику

## Библиотеки

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import animation

from IPython.display import HTML


from scipy.stats import beta, norm

## Сопряженное распределение

В чем проблемы данной формулы байеса?
$$
p\bigr(w|X\bigr) = \frac{p\bigr(w\bigr)p\bigr(X|w\bigr)}{p\bigr(X\bigr)}
$$

В вычислении можно хитрить, если знать, что апотериорное распределение $p\bigr(w|X\bigr)$ принадлежит тому же семейству распределения, что и априорное распределение $p\bigr(w\bigr),$ но с другими параметрами.

Например (очень часто используемый пример), если известно, что априорное и апостериорное распределения принадлежат нормальному распределению, то тогда много выкладок упрощается и получается элегантное решения задачи машиного обучения.

В случае, если семейства априорного и апостериорного распределения совпадают, то тогда, распределение $p\bigr(w\bigr)$ называется сопряженным к правдоподобию $p\bigr(X|w\bigr).$


[В википедии](https://ru.wikipedia.org/w/index.php?title=Сопряжённое_априорное_распределение&stable=1) приведена таблица основных сопряженных распределений.

Нам понадобится следующие:

1. Для Биномиальной функции правдоподобия сопряженным семейством является Бета распределение (не зря мы его уже использовали сегодня:)) )
2. Для Нормальной функции правдоподобия сопряженным семеством является Нормальное распределение.

| Функция правд. | Параметр | Соп. семей | Парам. апр. рас. | Парам. апост. распр.                           |
|----------------|----------|------------|------------------|------------------------------------------------|
|----------------|----------|------------|------------------|------------------------------------------------|
| Биномиальное   | p        | Бета       | $\alpha, \beta$  | $\alpha +\sum x_i, \beta + \sum N_i -\sum x_i$ |
| Нормальное     | $\mu$    | Нормальное |$\mu_0,\sigma^2_0$| $\left(\frac{\mu_0}{\sigma^2_0} + \frac{\sum x_i}{\sigma^2}\right)\cdot\left(\frac{1}{\sigma_0^2} + \frac{n}{\sigma^2}\right)^{-1}$, $\left(\frac{1}{\sigma_0^2} + \frac{n}{\sigma^2}\right)^{-1}$ |

## Синтетический пример

Рассмотрим монетку только вышедшую из конвеера. 

In [ ]:
x = np.linspace(0, 1, 100)
plt.plot(x, beta(2,2).pdf(x))

plt.grid()
plt.show()

### Случай, когда выпускаются ''честные монетки'' с p=0.5

In [ ]:
P = 0.5

def update_plot(frame_number, ns, plot):
    rs = np.random.RandomState(42)
    X = rs.choice([0,1], size=ns[frame_number], p=[1-P, P])
    x = np.linspace(0, 1, 100)
    plot[0] = ax.plot(
        x, beta(
            2 + X.sum(), 
            2 + ns[frame_number] - X.sum()).pdf(x))

fig = plt.figure(figsize=(8.0, 4.0), frameon=False)

ax = fig.add_subplot(111)

plot = [None, None, None]

plt.autoscale(tight=True)
fps = 1
frn = 100
ani = animation.FuncAnimation(
    fig, update_plot, frn, fargs=(np.arange(2*frn, step=2), plot), interval=1000/fps, blit=False, )

plt.grid()
plt.show()

In [ ]:
HTML(ani.to_jshtml())

### Случай, когда выпускаются ''счастливые монетки'' с p=0.9

In [ ]:
P = 0.9

def update_plot(frame_number, ns, plot):
    rs = np.random.RandomState(42)
    X = rs.choice([0,1], size=ns[frame_number], p=[1-P, P])
    x = np.linspace(0, 1, 100)
    plot[0] = ax.plot(
        x, beta(
            2 + X.sum(), 
            2 + ns[frame_number] - X.sum()).pdf(x))

fig = plt.figure(figsize=(8.0, 4.0), frameon=False)

ax = fig.add_subplot(111)

plot = [None, None, None]

plt.autoscale(tight=True)
fps = 1
frn = 100
ani = animation.FuncAnimation(
    fig, update_plot, frn, fargs=(np.arange(2*frn, step=2), plot), interval=1000/fps, blit=False, )

plt.grid()
plt.show()

In [ ]:
HTML(ani.to_jshtml())

## Модель машинного обучения

Рассмотрим синтетические данные сгенерированные следующим образом:
$$
y = wx + \varepsilon,
$$
то есть у нас один параметр, причем $\varepsilon \sim \mathcal{N}\bigr(0, \sigma^2\bigr)$.

В качестве априорного распределения расмотрим $p\bigr(w\bigr) = \mathcal{N}\bigr(0, \sigma_0^2\bigr).$
Для задачи регресии очевидно, что правдоподобие являтся также из нормального семейства, в нашем случае~$\mathcal{N}\bigr(0, \sigma^2\bigr).$

### Сильно ограниченная модель: $\sigma_0=0.1$

In [ ]:
sigma = 1
sigma_0 = 0.1

def update_plot(frame_number, ns, plot):
    rs = np.random.RandomState(42)
    
    X = rs.randn(ns[frame_number])
    y = 3*X + rs.randn(ns[frame_number])
    
    x = np.linspace(-5, 5, 1000)
    new_sigma = 1/(1/(sigma_0**2) + (X*X).sum()/(sigma**2))
    new_mu = new_sigma*(((X*y).sum())/(sigma**2))
    plot[0] = ax.plot(
        x, norm(
            new_mu, 
            new_sigma).pdf(x))

fig = plt.figure(figsize=(8.0, 4.0), frameon=False)

ax = fig.add_subplot(111)

plot = [None, None, None]

plt.autoscale(tight=True)
fps = 1
frn = 100
ani = animation.FuncAnimation(
    fig, update_plot, frn, fargs=(np.arange(2*frn, step=2), plot), interval=1000/fps, blit=False, )

plt.grid()
plt.show()

In [ ]:
HTML(ani.to_jshtml())

### Менее ограниченная модель: $\sigma_0=1$

In [ ]:
sigma = 1
sigma_0 = 1

def update_plot(frame_number, ns, plot):
    rs = np.random.RandomState(42)
    
    X = rs.randn(ns[frame_number])
    y = 3*X + rs.randn(ns[frame_number])
    
    x = np.linspace(-5, 5, 1000)
    new_sigma = 1/(1/(sigma_0**2) + (X*X).sum()/(sigma**2))
    new_mu = new_sigma*(((X*y).sum())/(sigma**2))
    plot[0] = ax.plot(
        x, norm(
            new_mu, 
            new_sigma).pdf(x))

fig = plt.figure(figsize=(8.0, 4.0), frameon=False)

ax = fig.add_subplot(111)

plot = [None, None, None]

plt.autoscale(tight=True)
fps = 1
frn = 100
ani = animation.FuncAnimation(
    fig, update_plot, frn, fargs=(np.arange(2*frn, step=2), plot), interval=1000/fps, blit=False, )

plt.grid()
plt.show()

In [ ]:
HTML(ani.to_jshtml())

### Более размазанная модель: $\sigma=10$

In [ ]:
sigma = 6
sigma_0 = 1

def update_plot(frame_number, ns, plot):
    rs = np.random.RandomState(42)
    
    X = rs.randn(ns[frame_number])
    y = 3*X + rs.randn(ns[frame_number])
    
    x = np.linspace(-5, 5, 1000)
    new_sigma = 1/(1/(sigma_0**2) + (X*X).sum()/(sigma**2))
    new_mu = new_sigma*(((X*y).sum())/(sigma**2))
    plot[0] = ax.plot(
        x, norm(
            new_mu, 
            new_sigma).pdf(x))

fig = plt.figure(figsize=(8.0, 4.0), frameon=False)

ax = fig.add_subplot(111)

plot = [None, None, None]

plt.autoscale(tight=True)
fps = 1
frn = 100
ani = animation.FuncAnimation(
    fig, update_plot, frn, fargs=(np.arange(3*frn, step=3), plot), interval=1000/fps, blit=False, )

plt.grid()
plt.show()

In [ ]:
HTML(ani.to_jshtml())